In [1]:
import os
import pymysql
import pandas as pd
from dotenv import load_dotenv, find_dotenv

In [2]:
env_path = find_dotenv()

load_dotenv(env_path)
print(os.getenv('HOST'))

aurora-staging-instance-1-ap-southeast-2a.cnhyotkoyfuh.ap-southeast-2.rds.amazonaws.com


In [3]:
import os
import pymysql
import pandas as pd

conn = pymysql.connect(
    host=os.getenv('HOST'),
    port=int(3306),
    user=os.getenv('USER_DB'),
    passwd=os.getenv('PASSWORD_DB'),
    db=os.getenv('DB'),
    charset='utf8mb4')

sub_df = pd.read_sql_query("SELECT * FROM stripe_subscription;", conn)

/Users/jebzhang/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
sub_df.head(5)

,id,Customer ID,Plan,Product,Quantity,Interval,Amount,Status,Created (UTC),Start (UTC),...,Coupon,Tax Percent,Canceled At (UTC),Cancel At Period End,Ended At (UTC),Exclusive Tax Rate,Inclusive Tax Rate,note (metadata),cancellation_reason (metadata),trialVersion (metadata)
0,sub_1Ja6DkFhaHFkXn0eWMa7rc5w,cus_KEZMuziMQwWZp6,nz-subscription-quarterly-100,Ausmed NZ Subscription,1,month,8696,incomplete_expired,2021-09-15 22:04,2021-09-15 22:04,...,,15.0,2021-09-16 21:05,false,2021-09-16 21:05,15.0,,,,
1,sub_1Ja7IGFhaHFkXn0eMcY1ZsgX,cus_KEaTUNnv66lLey,subscription-quarterly-99,Ausmed Subscription,1,month,9000,active,2021-09-15 23:13,2021-09-15 23:13,...,,10.0,,false,,10.0,,,,
2,sub_1Ja7YRFhaHFkXn0enD9ktwBG,cus_KEaiyp1QVwA9Em,subscription-quarterly-99,Ausmed Subscription,1,month,9000,canceled,2021-09-15 23:30,2021-09-18 23:35,...,,10.0,2021-09-18 23:35,true,2021-12-15 23:30,10.0,,,,
3,sub_1Ja8ovFhaHFkXn0eklNYSNB1,cus_KEc2mariQlALve,subscription-quarterly-99,Ausmed Subscription,1,month,9000,active,2021-09-16 00:51,2021-09-16 00:51,...,,10.0,,false,,10.0,,,,
4,sub_1Ja9SuFhaHFkXn0eqr5g4MeN,cus_HIzxlqwoIncy9j,subscription-quarterly-99,Ausmed Subscription,1,month,9000,active,2021-09-16 01:32,2021-09-16 01:32,...,,10.0,,false,,10.0,,,,


In [5]:
sub_df.columns

Index(['id', 'Customer ID', 'Plan', 'Product', 'Quantity', 'Interval',
       'Amount', 'Status', 'Created (UTC)', 'Start (UTC)', 'Start Date (UTC)',
       'Current Period Start (UTC)', 'Current Period End (UTC)',
       'Trial Start (UTC)', 'Trial End (UTC)', 'Application Fee Percent',
       'Coupon', 'Tax Percent', 'Canceled At (UTC)', 'Cancel At Period End',
       'Ended At (UTC)', 'Exclusive Tax Rate', 'Inclusive Tax Rate',
       'note (metadata)', 'cancellation_reason (metadata)',
       'trialVersion (metadata)'],
      dtype='object')

In [6]:
sub_df['Start Date (UTC)'].dtypes
sub_df.dtypes

id                                object
Customer ID                       object
Plan                              object
Product                           object
Quantity                           int64
Interval                          object
Amount                             int64
Status                            object
Created (UTC)                     object
Start (UTC)                       object
Start Date (UTC)                  object
Current Period Start (UTC)        object
Current Period End (UTC)          object
Trial Start (UTC)                 object
Trial End (UTC)                   object
Application Fee Percent           object
Coupon                            object
Tax Percent                       object
Canceled At (UTC)                 object
Cancel At Period End              object
Ended At (UTC)                    object
Exclusive Tax Rate                object
Inclusive Tax Rate                object
note (metadata)                   object
cancellation_rea

In [7]:
#1. who is the composite of active users
#2. how long did it take for people to cancel and when did they sign up?
#3. Plan sizes

In [8]:
import numpy as np

plan_activity_pivot = pd.pivot_table(sub_df, values = ['Quantity'], index = ['Plan'], columns = ['Status'], aggfunc = np.sum)
plan_activity_pivot.sort_values(by='Plan')

Quantity                                       \
Status                          active canceled incomplete_expired past_due   
Plan                                                                          
ausmed-loyalty-annually-297        NaN      1.0                NaN      NaN   
ausmed-loyalty-monthly-24        183.0    116.0                NaN      NaN   
ausmed-loyalty-monthly-297         NaN      5.0                NaN      NaN   
ausmed-pro-annually-360           73.0     87.0                NaN      1.0   
ausmed-pro-monthly-30            506.0   1788.0                NaN      9.0   
nz-subscription-annually-400       6.0      3.0                NaN      NaN   
nz-subscription-monthly-40         NaN      7.0                NaN      NaN   
nz-subscription-quarterly-100     60.0     77.0                3.0      2.0   
subscription-annually-297          NaN      3.0                NaN      NaN   
subscription-annually-330        251.0     25.0                9.0      2.0   
subscription-annually-360       1147.0    902.0                1.0     40.0   
subscription-annually-396        308.0     43.0                8.0     14.0   
subscription-monthly-24          505.0    290.0                NaN      4.0   
subscription-monthly-30          188.0    167.0                NaN      3.0   
subscription-monthly-33           34.0     36.0                3.0      5.0   
subscription-monthly-36          412.0   1504.0                3.0     22.0   
subscription-monthly-45           67.0    309.0                1.0      7.0   
subscription-quarterly-90       8239.0  12201.0               28.0    374.0   
subscription-quarterly-99       1800.0    968.0              134.0    119.0   
uk-subscription-annually-200       1.0      1.0                NaN      NaN   
uk-subscription-monthly-20         NaN      2.0                NaN      NaN   
uk-subscription-quarterly-50      34.0     87.0                2.0      8.0   
us-subscription-annually-229       7.0      4.0                NaN      1.0   
us-subscription-monthly-24         1.0     10.0                NaN      NaN   
us-subscription-quarterly-59      73.0    189.0               26.0     15.0   

                                        
Status                        trialing  
Plan                                    
ausmed-loyalty-annually-297        NaN  
ausmed-loyalty-monthly-24          NaN  
ausmed-loyalty-monthly-297         NaN  
ausmed-pro-annually-360            NaN  
ausmed-pro-monthly-30              NaN  
nz-subscription-annually-400       NaN  
nz-subscription-monthly-40         NaN  
nz-subscription-quarterly-100      1.0  
subscription-annually-297          NaN  
subscription-annually-330          7.0  
subscription-annually-360          2.0  
subscription-annually-396          2.0  
subscription-monthly-24            NaN  
subscription-monthly-30            NaN  
subscription-monthly-33            NaN  
subscription-monthly-36           26.0  
subscription-monthly-45            NaN  
subscription-quarterly-90         23.0  
subscription-quarterly-99          1.0  
uk-subscription-annually-200       NaN  
uk-subscription-monthly-20         NaN  
uk-subscription-quarterly-50       NaN  
us-subscription-annually-229       NaN  
us-subscription-monthly-24         NaN  
us-subscription-quarterly-59       3.0

In [9]:
sub_df[sub_df['Status'] == 'active']['Plan'].value_counts()

subscription-quarterly-90        8239
subscription-quarterly-99        1800
subscription-annually-360        1147
ausmed-pro-monthly-30             506
subscription-monthly-24           505
subscription-monthly-36           412
subscription-annually-396         308
subscription-annually-330         251
subscription-monthly-30           188
ausmed-loyalty-monthly-24         183
us-subscription-quarterly-59       73
ausmed-pro-annually-360            73
subscription-monthly-45            67
nz-subscription-quarterly-100      60
uk-subscription-quarterly-50       34
subscription-monthly-33            34
us-subscription-annually-229        7
nz-subscription-annually-400        6
us-subscription-monthly-24          1
uk-subscription-annually-200        1
Name: Plan, dtype: int64

In [10]:
sub_df['start_date'] = pd.to_datetime(sub_df['Start Date (UTC)'])
sub_df['end_date']= pd.to_datetime(sub_df['Canceled At (UTC)'])
sub_df['active_days']=(sub_df['end_date']-sub_df['start_date']).dt.days

In [11]:
canceled_sub = sub_df[sub_df['Status'] =='canceled'][['Plan','active_days']]
canceled_sub

,Plan,active_days
2,subscription-quarterly-99,3.0
5,subscription-quarterly-99,243.0
8,subscription-quarterly-99,115.0
9,subscription-quarterly-99,262.0
10,subscription-quarterly-99,262.0
...,...,...
33612,subscription-quarterly-99,216.0
33621,subscription-quarterly-99,117.0
33622,subscription-quarterly-99,83.0
33625,subscription-quarterly-99,146.0


In [12]:
import numpy as np

plan_cancel_pivot = pd.pivot_table(canceled_sub, values = ['active_days'], index = ['Plan'], aggfunc = np.mean)
plan_cancel_pivot

,active_days
Plan,
ausmed-loyalty-annually-297,886.000000
ausmed-loyalty-monthly-24,770.155172
ausmed-loyalty-monthly-297,487.400000
ausmed-pro-annually-360,991.321839
ausmed-pro-monthly-30,558.376398
nz-subscription-annually-400,431.333333
nz-subscription-monthly-40,224.571429
nz-subscription-quarterly-100,301.467532
subscription-annually-297,166.666667


In [13]:
sub_df[sub_df['Status']!= "active"][sub_df["Plan"] == "subscription-annually-330"][["Start Date (UTC)","Canceled At (UTC)","active_days"]].head(5)

/var/folders/t8/6sllmnn55m57p9d8z7w2ssrh0000gn/T/ipykernel_50840/754013729.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  sub_df[sub_df['Status']!= "active"][sub_df["Plan"] == "subscription-annually-330"][["Start Date (UTC)","Canceled At (UTC)","active_days"]].head(5)


,Start Date (UTC),Canceled At (UTC),active_days
1794,2022-04-13 02:35,2022-04-19 23:46,6.0
2076,2022-04-19 04:05,2022-04-23 13:20,4.0
2269,2022-04-23 15:50,2022-04-28 03:57,4.0
2364,2022-04-26 05:36,2022-05-02 00:12,5.0
2568,2022-04-28 05:54,2022-05-02 13:21,4.0


In [14]:
canceled_sub['Plan'].unique()

array(['subscription-quarterly-99', 'us-subscription-quarterly-59',
       'uk-subscription-quarterly-50', 'subscription-quarterly-90',
       'subscription-annually-396', 'subscription-monthly-33',
       'subscription-monthly-36', 'ausmed-pro-monthly-30',
       'nz-subscription-quarterly-100', 'subscription-monthly-30',
       'subscription-monthly-45', 'subscription-annually-360',
       'subscription-annually-330', 'ausmed-pro-annually-360',
       'ausmed-loyalty-monthly-24', 'ausmed-loyalty-monthly-297',
       'subscription-monthly-24', 'subscription-annually-297',
       'ausmed-loyalty-annually-297', 'nz-subscription-monthly-40',
       'us-subscription-monthly-24', 'us-subscription-annually-229',
       'uk-subscription-monthly-20', 'uk-subscription-annually-200',
       'nz-subscription-annually-400'], dtype=object)

In [15]:
# canceled_sub['plan_cleaned'] = canceled_sub['Plan'].str.strip("uk-subscription-").str.strip("nz-subscription-").str.strip("us-subscription-").\
# str.strip("ausmed-pro-").str.strip("ausmed-loyalty-").str.strip("subscription-")

In [16]:
def simple_period(x):
    if "quarterly" in x:
        y = "quarterly"
    elif "monthly" in x:
        y = "monthly"
    elif "annually" in x:
        y = "annually"
    return y

canceled_sub['plan_cleaned'] = canceled_sub['Plan'].apply(lambda x: simple_period(x))
canceled_sub_new = canceled_sub[['active_days','plan_cleaned']]
canceled_sub_new
                                

,active_days,plan_cleaned
2,3.0,quarterly
5,243.0,quarterly
8,115.0,quarterly
9,262.0,quarterly
10,262.0,quarterly
...,...,...
33612,216.0,quarterly
33621,117.0,quarterly
33622,83.0,quarterly
33625,146.0,quarterly


In [17]:
import numpy as np

plan_cancel_pivot_new = pd.pivot_table(canceled_sub_new, values = ['active_days'], index = ['plan_cleaned'], aggfunc = np.mean)
plan_cancel_pivot_new

,active_days
plan_cleaned,
annually,607.932647
monthly,468.519603
quarterly,361.392915


In [49]:
from datetime import datetime

def date_convert(date_to_convert):
    return datetime.strptime(str(date_to_convert), "%Y-%m-%d %H:%M:%S").month

sub_df['month']= sub_df['start_date'].apply(date_convert)
sub_df['plan_cleaned'] = sub_df['Plan'].apply(lambda x: simple_period(x))

KeyError: 'start_date'

In [19]:
sub_df[['plan_cleaned','Status']].value_counts()

plan_cleaned  Status            
quarterly     canceled              13522
              active                10206
monthly       canceled               4234
              active                 1896
annually      active                 1793
              canceled               1069
quarterly     past_due                518
              incomplete_expired      193
annually      past_due                 58
monthly       past_due                 50
quarterly     trialing                 28
monthly       trialing                 26
annually      incomplete_expired       18
              trialing                 11
monthly       incomplete_expired        7
dtype: int64

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.histplot(sub_df['month'])

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
canceled_sub1 = sub_df[sub_df['Status'] =='canceled']
canceled_sub1['month_finish'] = canceled_sub1['end_date'].apply(date_convert)

sns.histplot(canceled_sub1['month_finish'])

# Plan_Type

In [50]:
import os
import pymysql
import pandas as pd

conn = pymysql.connect(
    host=os.getenv('HOST'),
    port=int(3306),
    user=os.getenv('USER_DB'),
    passwd=os.getenv('PASSWORD_DB'),
    db=os.getenv('DB'),
    charset='utf8mb4')

sub_df = pd.read_sql_query("""select userID,
                                       stripeCustID,
                                       stripe_subscription.id,
                                       stripe_subscription.Plan,
                                       stripe_subscription.Quantity,
                                       stripe_subscription.Amount,
                                       stripe_subscription.Status,
                                       stripe_subscription.`Start (UTC)`,
                                       stripe_subscription.`Canceled At (UTC)`,
                                       stripe_subscription.`Ended At (UTC)`,
                                       stripe_subscription.`cancellation_reason (metadata)`
                             FROM user
                                inner join stripe_subscription 
                                on user.stripeCustID = stripe_subscription.`Customer ID` ;""", conn)

/Users/jebzhang/.pyenv/versions/3.8.13/envs/customerclustering/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [51]:
sub_df

,userID,stripeCustID,id,Plan,Quantity,Amount,Status,Start (UTC),Canceled At (UTC),Ended At (UTC),cancellation_reason (metadata)
0,00001d9c-5de5-4ed2-9ecc-98341286765a,cus_DW2cclQzaW25U8,sub_HDj18sYVhLg43x,subscription-quarterly-90,1,8182,canceled,2020-05-05 03:35,2021-12-17 05:56,2021-12-17 05:56,
1,0001c897-b1fe-40e0-afef-6a667edf41f7,cus_CFStnWtnKt01vA,sub_CFSxMGPdDXRoZI,ausmed-pro-annually-360,1,32727,active,2018-02-02 06:27,,,
2,00036835-5926-476a-bd8a-65c39be4e6cc,cus_JZ4BVudyE1rQcn,sub_JZQDYuZNHL8uLH,subscription-quarterly-99,1,9000,canceled,2021-05-29 01:53,2021-12-17 04:55,2021-12-17 04:55,
3,0007e271-2af0-4dc4-9b27-b485586320b9,cus_GVh1jLQXQiluuu,sub_Gk7DGQ3EFtl4TW,subscription-quarterly-90,1,8182,canceled,2022-03-09 02:03,2022-03-09 02:03,2022-05-16 02:35,
4,0008e1bc-589a-4bf3-ae16-ace555566578,cus_AkvwV7AwHostTr,sub_EyT38vAmHzgz1A,subscription-quarterly-90,1,8182,canceled,2020-06-27 00:28,2020-06-27 00:28,2020-07-29 09:50,
...,...,...,...,...,...,...,...,...,...,...,...
31474,fff5b6ec-0f17-4c0f-acdb-caf5a54d440c,cus_AlI7ExGfEYx9BI,sub_ExZPLwSbEJwSSR,subscription-quarterly-90,1,8182,active,2019-04-27 00:19,,,
31475,fff61169-6743-4b75-b100-3603d1c02082,cus_HBQubDAIn90yiH,sub_HBQwip1NdPt0LB,subscription-quarterly-90,1,8182,active,2020-04-29 00:46,,,
31476,fff6edf1-916e-496e-b811-ec8a60b4dc3f,cus_Htf9RoTV17oNz6,sub_JvBvofOtuOygtj,subscription-quarterly-99,1,9000,active,2021-07-26 04:36,,,
31477,fff8d7d8-d0a5-4fb1-9e1f-a6063f16c28d,cus_IjezQrGVvBmqui,sub_JL8Z8pK2S3kKJz,subscription-quarterly-90,1,8182,canceled,2021-08-25 03:46,2021-08-25 03:46,2021-10-20 22:43,


In [40]:
sub_df['cancellation_reason (metadata)'].value_counts()

                                                                    31453
Found this duplicated subscription when processing subscription.       26
Name: cancellation_reason (metadata), dtype: int64

In [87]:
#doubt: there are  over 307k users, 212k cust_sub account per user table;
#there are only 31k unique cust_ID in the stripesubsrciption table;
#do we have enough plan data to make this a viable feature;
#there are also dupe subscriptions in the stripe_subscription data, ie 'Customer ID` ='cus_CwbLsaFBPovaL4'

#define plan type based on the plan name whether there is periodic wording in it.

def plan_type(x):
    if "quarterly" in x:
        y = "quarterly"
    elif "monthly" in x:
        y = "monthly"
    elif "annually" in x:
        y = "annually"
    return y

sub_df['plan_type'] = sub_df['Plan'].apply(lambda x: plan_type(x))
sub_df

plan_df = pd.pivot_table(sub_df, values = ['start_date'],\
                                    index = ['userID','plan_type'], aggfunc = np.max).reset_index()
plan_df


,userID,plan_type,start_date
0,00001d9c-5de5-4ed2-9ecc-98341286765a,quarterly,2020-05-05 03:35:00
1,0001c897-b1fe-40e0-afef-6a667edf41f7,annually,2018-02-02 06:27:00
2,00036835-5926-476a-bd8a-65c39be4e6cc,quarterly,2021-05-29 01:53:00
3,0007e271-2af0-4dc4-9b27-b485586320b9,quarterly,2022-03-09 02:03:00
4,0008e1bc-589a-4bf3-ae16-ace555566578,quarterly,2020-06-27 00:28:00
...,...,...,...
29945,fff5b6ec-0f17-4c0f-acdb-caf5a54d440c,quarterly,2019-04-27 00:19:00
29946,fff61169-6743-4b75-b100-3603d1c02082,quarterly,2020-04-29 00:46:00
29947,fff6edf1-916e-496e-b811-ec8a60b4dc3f,quarterly,2021-07-26 04:36:00
29948,fff8d7d8-d0a5-4fb1-9e1f-a6063f16c28d,quarterly,2021-08-25 03:46:00


In [63]:
sub_no_end_date_df = sub_df[sub_df['Ended At (UTC)'] == '']
sub_no_end_date_df['Status'].value_counts()

active      12850
past_due      571
trialing       27
Name: Status, dtype: int64

In [64]:
datetime.today().strftime('%Y-%m-%d')

'2022-08-14'

In [80]:
from datetime import datetime

sub_df['start_date'] = pd.to_datetime(sub_df['Start (UTC)'])
sub_df['cancel_date']= pd.to_datetime(sub_df['Canceled At (UTC)'])

def end_date(date):
    if date == '':
        date = datetime.today().strftime('%Y-%m-%d')
    else:
        date
    return date

sub_df['ended_date'] = pd.to_datetime(sub_df['Ended At (UTC)'].apply(lambda x: end_date(x)))
sub_df['active_days']=(sub_df['ended_date']-sub_df['start_date']).dt.days
active_df = pd.pivot_table(sub_df, values = ['active_days'],\
                                    index = ['userID'], aggfunc = np.sum).reset_index()
active_df


,userID,active_days
0,00001d9c-5de5-4ed2-9ecc-98341286765a,591
1,0001c897-b1fe-40e0-afef-6a667edf41f7,1653
2,00036835-5926-476a-bd8a-65c39be4e6cc,202
3,0007e271-2af0-4dc4-9b27-b485586320b9,68
4,0008e1bc-589a-4bf3-ae16-ace555566578,32
...,...,...
29111,fff5b6ec-0f17-4c0f-acdb-caf5a54d440c,1204
29112,fff61169-6743-4b75-b100-3603d1c02082,836
29113,fff6edf1-916e-496e-b811-ec8a60b4dc3f,383
29114,fff8d7d8-d0a5-4fb1-9e1f-a6063f16c28d,56


In [75]:
end_date(sub_df['start_date'][0])

Timestamp('2020-05-05 03:35:00')